In [29]:
import os
import pandas as pd
from oct_converter.readers import E2E

# Step 1: Load the CSV file, assuming the first row is the correct header
csv_file_path = 'AMD_Artificial_intelligence.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path, header=1)

# Step 2: Filter for rows where 'Name' is 'Acél Ede'
df_acel_ede = df[df['Name'] == 'Bagó Lajosné']

# Step 3: Function to extract acquisition date from the .e2e file
def get_e2e_acquisition_dates(folder_path):
    acquisition_dates = set()
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.E2E'):
            file_path = os.path.join(folder_path, filename)
            try:
                fds = E2E(file_path)
                oct_volume = fds.read_oct_volume()
                tmp = oct_volume.pop()
                acquisition_dates.add(pd.to_datetime(tmp.acquisition_date))
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    
    return acquisition_dates

# Step 4: Get all acquisition dates from the folder
folder_path = 'D:/AMD Artificial Intelligence Research/Balogh Benjaminne'  # Replace with your actual folder path
acquisition_dates = get_e2e_acquisition_dates(folder_path)

# Step 5: Check if each 'Scan Date' from the CSV is in the acquisition dates
missing_dates = []

for index, row in df_acel_ede.iterrows():
    try:
        # Convert 'Scan Date' to datetime, handling different formats
        scan_date = pd.to_datetime(row['Scan Date'], errors='coerce')  # Automatically infer the format
        if pd.isna(scan_date):
            print(f"Warning: Unable to parse 'Scan Date' value: {row['Scan Date']}")
            continue
        
        if scan_date not in acquisition_dates:
            missing_dates.append(scan_date)
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Step 6: Output missing dates
if missing_dates:
    print("The following 'Scan Date' values are not found in any .e2e files:")
    for date in missing_dates:
        print(date.strftime('%Y-%m-%d'))
else:
    print("All 'Scan Date' values are found in the .e2e files.")


All 'Scan Date' values are found in the .e2e files.
